<a href="https://colab.research.google.com/github/katewall/2023class_size_predictions/blob/main/PHM1690_school_prediction_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import LeaveOneOut, cross_val_score
import statsmodels.api as sm
from scipy.stats import shapiro
!pip install kaleido
import kaleido
import plotly.graph_objects as go

path = '/content/drive/MyDrive/AA: Writing/school challenge/falldf.csv'
falldf = pd.read_csv(path)
falldf

,Term,Year,PHM_1690,PH_1700,PH_1975,PH_1976,admitted.students.BaDS.MS,accepted.offer.of.admission.BaDS.MS,enrolled.students.BaDS.MS,admitted.students.BaDS.PHD,...,pre_accept_MPCH,pre_enroll_BaDS,pre_enroll_EHGES,pre_enroll_HPBS,pre_enroll_MPCH,Pct_ut,Pct_am,Grd_fem,Grd_mal,PHM_1690_raw
0,2163.0,2016,153.0,49.0,9999.0,9999.0,45.0,11.0,11.0,33.0,...,18.0,2.0,21.0,13.0,15.0,0.3753,0.3802,1750692,1221563,153.0
1,2173.0,2017,246.0,57.0,36.0,9999.0,33.0,13.0,11.0,43.0,...,18.0,5.0,19.0,9.0,16.0,0.4043,0.4016,1785060,1220055,246.0
2,2183.0,2018,207.0,92.0,36.0,9999.0,48.0,13.0,8.0,66.0,...,16.0,3.0,24.0,9.0,15.0,0.4240,0.4130,1818576,1216466,207.0
3,2193.0,2019,225.0,79.0,64.0,9999.0,42.0,15.0,12.0,72.0,...,15.0,4.0,24.0,19.0,10.0,0.4418,0.4244,1858200,1214439,225.0
4,2203.0,2020,302.0,96.0,58.0,9999.0,43.0,18.0,14.0,74.0,...,23.0,4.0,29.0,27.0,22.0,0.4807,0.4238,1922913,1219938,302.0
5,2213.0,2021,313.0,111.0,91.0,9999.0,63.0,28.0,17.0,171.0,...,19.0,4.0,43.0,15.0,17.0,0.4865,0.4447,1967393,1244038,313.0
6,2223.0,2022,217.0,108.0,50.0,9999.0,60.0,16.0,14.0,121.0,...,12.0,2.0,29.0,11.0,9.0,0.4955,0.4463,1908707,1182312,217.0
7,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,1.0,14.0,6.0,7.0,NaN,NaN,1893388,1158767,NaN


In [8]:
possible_predictors = ['Year', 'PH_1975', 'PH_1976',
       'admitted.students.BaDS.MS', 'accepted.offer.of.admission.BaDS.MS',
       'enrolled.students.BaDS.MS', 'admitted.students.BaDS.PHD',
       'accepted.offer.of.admission.BaDS.PHD', 'enrolled.students.BaDS.PHD',
       'admitted.students.BaDS.MPH', 'accepted.offer.of.admission.BaDS.MPH',
       'enrolled.students.BaDS.MPH', 'admitted.students.EHGES.MS',
       'accepted.offer.of.admission.EHGES.MS', 'enrolled.students.EHGES.MS',
       'admitted.students.EHGES.PHD', 'accepted.offer.of.admission.EHGES.PHD',
       'enrolled.students.EHGES.PHD', 'admitted.students.EHGES.MPH',
       'accepted.offer.of.admission.EHGES.MPH', 'enrolled.students.EHGES.MPH',
       'admitted.students.HPBS.PHD', 'accepted.offer.of.admission.HPBS.PHD',
       'enrolled.students.HPBS.PHD', 'admitted.students.HPBS.MPH',
       'accepted.offer.of.admission.HPBS.MPH', 'enrolled.students.HPBS.MPH',
       'admitted.students.MPCH.PHD', 'accepted.offer.of.admission.MPCH.PHD',
       'enrolled.students.MPCH.PHD', 'admitted.students.MPCH.MPH',
       'accepted.offer.of.admission.MPCH.MPH', 'enrolled.students.MPCH.MPH',
       'accept_BaDS', 'accept_EHGES',
       'accept_HPBS', 'accept_MPCH', 'enroll_BaDS', 'enroll_EHGES',
       'enroll_HPBS', 'enroll_MPCH', 'pre_accept_BaDS', 'pre_accept_EHGES',
       'pre_accept_HPBS', 'pre_accept_MPCH', 'pre_enroll_BaDS',
       'pre_enroll_EHGES', 'pre_enroll_HPBS', 'pre_enroll_MPCH','Pct_ut',
       'Pct_am', 'Grd_fem', 'Grd_mal']

In [9]:
possible_predictors = ['Year', 'pre_accept_BaDS', 'pre_accept_EHGES',
       'pre_accept_HPBS', 'pre_accept_MPCH', 'pre_enroll_BaDS',
       'pre_enroll_EHGES', 'pre_enroll_HPBS', 'pre_enroll_MPCH','Pct_ut',
       'Pct_am', 'Grd_fem', 'Grd_mal']

# Leave One Out CV

In [10]:
df_loo = falldf[falldf['Year'] != 2023]
y = np.array(df_loo['PHM_1690']).reshape(-1,1)
loo = LeaveOneOut()
reg = linear_model.LinearRegression()
def loocv(model, df, y, possible_predictors):

  loo_dict = {}

  for i in possible_predictors:
    X = np.array(df.filter([i], axis=1)).reshape(-1,1)
    loo_dict[i] = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=loo)
  predictor_df = pd.DataFrame.from_dict(loo_dict, 'index')
  predictor_df['cv'] = -1 * predictor_df.sum(1) / df.shape[0]
  return predictor_df

predictor_df = loocv(reg, df_loo, y, possible_predictors)
predictor_df.sort_values('cv', ascending=True)[:10]

,0,1,2,3,4,5,6,cv
pre_accept_BaDS,-3765.334173,-272.893237,-148.910733,-19.802500,-3801.360285,-4260.528926,-666.986767,1847.973803
Grd_fem,-1842.805689,-4994.115582,-91.263613,-203.791453,-1414.880015,-659.892453,-3826.169667,1861.845496
pre_enroll_EHGES,-5253.123014,-4737.530081,-365.326028,-6.150578,-4169.257521,-200.052736,-1283.310367,2287.821475
pre_enroll_MPCH,-9968.674788,-0.352976,-1356.462798,-1049.562147,-1300.340978,-5493.584871,-1288.953924,2922.561783
pre_enroll_HPBS,-8299.601833,-2133.285156,-55.316406,-1608.126234,-1617.748062,-7493.496929,-27.614578,3033.598457
Pct_ut,-2982.238331,-3327.776565,-270.355723,-155.688623,-1969.071772,-3044.689883,-9936.993996,3098.116414
pre_accept_MPCH,-11403.727197,-6.882907,-541.873546,-85.845898,-1376.410000,-5506.512976,-2947.138481,3124.055858
pre_accept_HPBS,-8059.369964,-1802.354722,-16.391884,-2306.515970,-1940.095187,-7744.000000,-67.169565,3133.699613
pre_enroll_BaDS,-4200.660156,-6219.225922,-402.671111,-1450.917355,-3181.985537,-4887.280992,-1865.160156,3172.557318
pre_accept_EHGES,-7492.580573,-5263.835303,-188.124459,-0.002582,-4846.388967,-3289.489000,-1962.568408,3291.855613


In [11]:
toppreds = predictor_df.sort_values('cv', ascending=True)[:10].index.tolist()
print(toppreds)
df_loo[['Grd_fem','pre_accept_BaDS']].corr()

['pre_accept_BaDS', 'Grd_fem', 'pre_enroll_EHGES', 'pre_enroll_MPCH', 'pre_enroll_HPBS', 'Pct_ut', 'pre_accept_MPCH', 'pre_accept_HPBS', 'pre_enroll_BaDS', 'pre_accept_EHGES']


,Grd_fem,pre_accept_BaDS
Grd_fem,1.000000,0.461118
pre_accept_BaDS,0.461118,1.000000


In [12]:
X = np.array(df_loo.filter(['Year'], axis=1)).reshape(-1,1)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-5426.777778,-2905.21,-357.703214,-215.111111,-3615.669187,-4032.25,-15030.76,4511.925899


In [13]:
X = np.array(df_loo.filter(['Year','pre_accept_BaDS'], axis=1)).reshape(-1,2)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-1110.779461,-302.291553,-482.78797,-30.951785,-4741.763656,-2304.0,-3687.031368,1808.515113


In [14]:
X = np.array(df_loo.filter(['Year','Grd_fem'], axis=1)).reshape(-1,2)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-3342.558231,-4356.745138,-77.841449,-190.026231,-259.699535,-344.97734,-34.425026,1229.467564


In [15]:
X = np.array(df_loo.filter(['Year','pre_accept_BaDS','Grd_fem'], axis=1)).reshape(-1,3)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-1770.252548,-2902.711539,-180.374653,-70.234488,-3248.519067,-2531.358481,-1950.580605,1807.718769


In [16]:
X = np.array(df_loo.filter(['Grd_fem'], axis=1)).reshape(-1,1)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-1842.805689,-4994.115582,-91.263613,-203.791453,-1414.880015,-659.892453,-3826.169667,1861.845496


In [17]:
X = np.array(df_loo.filter(['pre_accept_BaDS'], axis=1)).reshape(-1,1)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-3765.334173,-272.893237,-148.910733,-19.8025,-3801.360285,-4260.528926,-666.986767,1847.973803


In [18]:
X = np.array(df_loo.filter(['Grd_fem','pre_accept_BaDS'], axis=1)).reshape(-1,2)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-904.951236,-1515.655807,-286.179743,-52.903615,-3551.631973,-1264.731641,-524.99077,1157.292112


# Model with Grd_fem and pre_accept_BaDS selected.

In [19]:
X = np.array(df_loo.filter(['Grd_fem','pre_accept_BaDS'], axis=1))
X = sm.add_constant(X)
y = np.array(df_loo['PHM_1690'])

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.908
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     19.65
Date:                Thu, 31 Aug 2023   Prob (F-statistic):            0.00854
Time:                        19:32:17   Log-Likelihood:                -29.190
No. Observations:                   7   AIC:                             64.38
Df Residuals:                       4   BIC:                             64.22
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -556.8843    214.122     -2.601      0.0

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 7 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [20]:
trace1 = go.Scatter(
  x= df_loo['Year'],
  y= results.resid,
  fill= 'none',
  opacity= 0.6,
  type= 'scatter',
  mode= 'markers',
  name= 'Residuals'
)

fig = go.Figure()
fig.add_trace(trace1)

fig.update_layout(
  title= 'PH 1690 Residuals',
  xaxis_title='Year',
  yaxis_title='Residuals',
  legend=dict(orientation='h')
)

fig.show()

In [21]:
shapiro(results.resid)

ShapiroResult(statistic=0.8608091473579407, pvalue=0.15393544733524323)

In [22]:
results.predict()

array([168.99442551, 225.14760326, 194.62957078, 231.18021334,
       320.3799898 , 294.00231046, 228.66588684])

# Final Prediction

In [23]:
falldf.filter(['Year','pre_accept_BaDS'], axis=1)

,Year,pre_accept_BaDS
0,2016,3.0
1,2017,5.0
2,2018,3.0
3,2019,4.0
4,2020,7.0
5,2021,5.0
6,2022,3.0
7,2023,2.0


In [24]:
falldf.filter(['pre_accept_BaDS'], axis=1)[0:7].mean()

pre_accept_BaDS    4.285714
dtype: float64

In [25]:
falldf[falldf['Year'] == 2023].filter(['Grd_fem','pre_accept_BaDS'])

,Grd_fem,pre_accept_BaDS
7,1893388,2.0


In [26]:
#X_new = np.array(falldf[falldf['Year'] == 2023].filter(['Grd_fem','pre_accept_BaDS'], axis=1)).reshape(-1,2)
X_new = np.concatenate((np.ones((1,1)),np.array([[1893388,4.285714]])), axis=1, dtype=float)
results.predict(X_new)

array([250.63612797])

# Plot Results

In [29]:
preds = falldf.filter(['Grd_fem','pre_accept_BaDS'], axis=1)
preds.loc[7,'pre_accept_BaDS'] = falldf.filter(['pre_accept_BaDS'], axis=1)[0:7].mean()[0]
X_preds = np.array(preds).reshape(-1,2)
X_preds = sm.add_constant(X_preds)
X_preds = results.predict(X_preds)

trace1 = go.Scatter(
  x= falldf['Year'],
  y= X_preds,
  fill= 'none',
  #fillcolor= '#5aabdd',
  opacity= 0.6,
  type= 'scatter',
  mode= 'lines',
  name= 'Predictions',
)

trace2 = go.Scatter(
  x= falldf['Year'],
  y= falldf['PHM_1690_raw'],
  fill= 'none',
  opacity= 0.6,
  type= 'scatter',
  mode= 'markers',
  marker = dict(size=14),
  name= 'Actuals',
)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
  title= 'PHM 1690 Class Size Predictions',
  xaxis_title='Year',
  yaxis_title='Headcount',
  legend=dict(orientation='h'))
fig.write_image('/content/fig1690_preds.png', engine='kaleido')
fig.show()

In [28]:
falldf.filter(['Year','Grd_fem','PHM_1690'], axis=1)

,Year,Grd_fem,PHM_1690
0,2016,1750692,153.0
1,2017,1785060,246.0
2,2018,1818576,207.0
3,2019,1858200,225.0
4,2020,1922913,302.0
5,2021,1967393,313.0
6,2022,1908707,217.0
7,2023,1893388,NaN
